In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import udf, col, concat_ws, collect_list, lit, log
import sys
import re
import pandas as pd

spark = SparkSession.builder.appName("ConstructivenessAnalysis").getOrCreate()

In [ ]:
construct = '/Users/ting/Documents/CMPT733/Project/SOCC/annotated/constructiveness/SFU_constructiveness_toxicity_corpus.csv'
df = spark.read.load(construct,
                     format='com.databricks.spark.csv', 
                     header='true')

In [ ]:
df_sub = df.select('article_id','comment_text', 'is_constructive','is_constructive:confidence','toxicity_level','toxicity_level:confidence',)

In [ ]:
import pandas as pd
expert_df = '/Users/ting/Documents/CMPT733/Project/sentimentAnalysis/ydata-ynacc-v1_0/ydata-ynacc-v1_0_expert_annotations.tsv'
expert_df = pd.read_csv(expert_df,  sep='\t')
expert_df = expert_df[['sdid','text','constructiveclass','sentiment','persuasiveness']].dropna()
print(expert_df.shape)
expert_df.head()

In [ ]:
expert_df['constructiveclass']  = expert_df['constructiveclass'].astype('category').cat.rename_categories([1,0]) #[Constructive, Not constructive]
expert_df['sentiment']  = expert_df['sentiment'].astype('category').cat.rename_categories([-1,0, 2, 1]) #[mixed, negative, neutral, positive]
expert_df['persuasiveness']  = expert_df['persuasiveness'].astype('category').cat.rename_categories([0,1])#[Not persuasive, Persuasive]

expert_df.head()

In [ ]:
# expert_df.to_csv('label_construct_sentiment_persuasive_train.csv')
# construct_train = "label_construct_sentiment_persuasive_train.csv"

# df = spark.read.load(construct_train,
#                      format='com.databricks.spark.csv', 
#                      header='true', 
#                      inferSchema='true')
# df.head()

In [1]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

p_stemmer = PorterStemmer()

def review_to_wordlist( review, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
#     words = [p_stemmer.stem(w) for w in words]
    #
    # 5. Return a list of words
#     return(words)
    if len(words) == 0:
        words = ['NULL']
    return( " ".join(words ))

udf_review = udf(lambda w: review_to_wordlist(w), StringType())
clean_df = df.select('sdid', udf_review(df.text).alias('cleaned_text'),'constructiveclass') 
clean_df.coalesce(1).write.csv('cleaned_label_construct_sentiment_persuasive_train', header = True)

In [ ]:
# clean_data = '/Users/ting/Documents/CMPT733/Project/sentimentAnalysis/cleaned_label_construct_sentiment_persuasive_train/part-00000-03d520ca-658f-4b9e-a783-cc39627d591d-c000.csv'
# clean_df = pd.read_csv(clean_data)
# clean_df.head()

In [2]:
import pandas as pd
construct_train = "label_construct_sentiment_persuasive_train.csv"
construct_train= pd.read_csv(construct_train)
construct_train.head()

,Unnamed: 0,sdid,text,constructiveclass,sentiment,persuasiveness
0,0,53971,"These things happen , Every job has its dangers.",1,0,0
1,1,53971,Sad to hear such a bad thing. Very dangerous j...,1,-1,0
2,2,53971,Yes..because too many houses in EU look like t...,1,2,0
3,3,135929,"I am frankly quite SICK of the phrase ""shoved ...",0,0,1
4,4,135929,"Ya, I always wonder why the conservatives are ...",0,2,0


In [3]:
num_reviews = construct_train["text"].size

clean_train_reviews = []

for i in range( 0, num_reviews ):
    clean_train_reviews.append( review_to_wordlist( construct_train["text"][i] ) )

/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/ting/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

In [5]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit( train_data_features, construct_train["constructiveclass"] )

In [6]:
test_data = '/Users/ting/Documents/CMPT733/Project/sentimentAnalysis/cleaned_comment/part-00000-803122b2-81f6-4428-8fc6-41962b3705d0-c000.csv'
test = pd.read_csv(test_data,quoting=3,error_bad_lines=False)
test.head()

b'Skipping line 197535: expected 3 fields, saw 25\n'


,article_id,comment_id,cleaned_text
0,10012655,33750cc126314c84b4babae99e97b347,think program needs work probably costly overs...
1,10012655,4ce8f60b0ddd442c8a3ac70c15feb954,offshoring reverse well union busting say good...
2,10012655,75900e8bd92c451491729551878a166d,spell exploitation disgusting practice sanctio...
3,10012655,ac49765f024640ae93e0913cdfbb4d48,tfws place economy canadians would leave home ...
4,10012655,c5b63fd3000e4306960411384e2999b2,temporary workers get paid tim horton rest com...


In [7]:
import numpy as np
clean_test_reviews = test["cleaned_text"]
test_data_features = vectorizer.transform(clean_test_reviews.values.astype('U'))
test_data_features = test_data_features.toarray()
predition = forest.predict(test_data_features)

predition = np.asarray(predition)
test['constructiv'] = predition
test.to_csv('comment_w_constructive_tfidfbw.csv')